# Fruits 360 Dataset

Creating a classifier of fruits utilizing the [fruits 360 dataset](https://www.kaggle.com/moltean/fruits) form kaggle. I had some issues with GPU utilization within Keras. So I recreated to an extent the Keras code below in PyTorch.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from keras.applications.xception import Xception
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, Activation, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import RMSprop, SGD
from keras import backend as K

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=10,       # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1,        # Randomly zoom image 
        width_shift_range=0.1,   # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True)    # randomly flip images

test_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    directory = '../data/fruits-360/Training',
    target_size = (64, 64),
    color_mode = 'rgb',
    batch_size = 32,
    class_mode = 'categorical',
    shuffle = True,
    seed = 42
)

test_generator = test_datagen.flow_from_directory(
    directory = '../data/fruits-360/Test',
    target_size = (64, 64),
    color_mode = 'rgb',
    batch_size = 1,
    class_mode = None,
    shuffle = False,
    seed = 42
)

In [ ]:
tf.logging.set_verbosity(tf.logging.ERROR)
tf_config = tf.ConfigProto(allow_soft_placement=False)
tf_config.gpu_options.allow_growth = True
s = tf.Session(config=tf_config)

K.set_session(s)

model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(5,5), padding='Same', activation ='relu', input_shape=(64,64,3)))
model.add(Conv2D(filters=32, kernel_size=(5,5), padding='Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=(3,3),padding='Same', activation ='relu'))
model.add(Conv2D(filters=64, kernel_size=(3,3),padding='Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(1048, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(103, activation="softmax"))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')


model.fit_generator(train_generator,
                    steps_per_epoch=53177,
                    epochs=30,
                    validation_data=test_generator,
                    validation_steps=8000,
                    workers=4)

# PyTorch Convolutional Neural Network

In [1]:
import torch
import torch.nn.functional as F
from torch import nn
import torch.optim as optim
from torch.autograd import Variable
from torch.optim import lr_scheduler
from torchvision import transforms, datasets

batch_size = 24

# Configuring transformations to be applied to the images to increase
# final accuracy.
data_transform = transforms.Compose([
        transforms.RandomResizedCrop(100),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ])

train_dataset = datasets.ImageFolder(root='../data/fruits-360/Training',
                                     transform=data_transform)

test_dataset = datasets.ImageFolder(root='../data/fruits-360/Test',
                                    transform=data_transform)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)

## Defining Neural Net

In [4]:
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        
        self.conv_layer = nn.Sequential(
             nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=0),
             nn.ReLU(),
             nn.Conv2d(32, 32, kernel_size=5, stride=1, padding=0),
             nn.ReLU(),
             nn.MaxPool2d(2),
             nn.Dropout(p=0.25),
             nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=0),
             nn.ReLU(),
             nn.Conv2d(64, 64, kernel_size=5, stride=1, padding=0),
             nn.ReLU(),
             nn.MaxPool2d(2),
             nn.Dropout(p=0.25)
        )
        
        self.fc_linear_layer = nn.Sequential(
             nn.Linear(64*19*19, 500),
             nn.ReLU(),
             nn.Linear(500, 250),
             nn.ReLU(),
             nn.Linear(250, 103)
        )
        
    def forward(self, x):
        x = self.conv_layer(x)
        x = x.view(-1, 64 * 19 * 19)
        x = self.fc_linear_layer(x)
        
        return x

## Initializing Model and Train/Eval Functions

In [7]:
model = ConvNet()

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
criterion = nn.CrossEntropyLoss()
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()

def train(epoch):
    model.train()
    exp_lr_scheduler.step()

    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        
        loss.backward()
        optimizer.step()
        
        if (batch_idx + 1)% 250 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, (batch_idx + 1) * len(data), len(train_loader.dataset),
                100. * (batch_idx + 1) / len(train_loader), loss))
            
def evaluate(data_loader):
    model.eval()
    loss = 0
    correct = 0
    
    for data, target in data_loader:
        data, target = Variable(data, requires_grad=True), Variable(target)
        if torch.cuda.is_available():
            data = data.cuda()
            target = target.cuda()
        
        output = model(data)
        
        loss += F.cross_entropy(output, target).data

        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()
        
    loss /= len(data_loader.dataset)
        
    print('\nAverage loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        loss, correct, len(data_loader.dataset),
        100. * correct / len(data_loader.dataset)))

## Running The Model

In [ ]:
n_epochs = 30

for epoch in range(n_epochs):
    train(epoch)
    evaluate(train_loader)

Train Epoch: 0 [6000/53177 (11%)]	Loss: 4.646219
Train Epoch: 0 [12000/53177 (23%)]	Loss: 4.416005
Train Epoch: 0 [18000/53177 (34%)]	Loss: 4.393569
Train Epoch: 0 [24000/53177 (45%)]	Loss: 4.136398
Train Epoch: 0 [30000/53177 (56%)]	Loss: 3.519601
Train Epoch: 0 [36000/53177 (68%)]	Loss: 2.453831
Train Epoch: 0 [42000/53177 (79%)]	Loss: 2.370917
Train Epoch: 0 [48000/53177 (90%)]	Loss: 2.319780

Average loss: 0.0874, Accuracy: 20191/53177 (37%)

Train Epoch: 1 [6000/53177 (11%)]	Loss: 1.473321
Train Epoch: 1 [12000/53177 (23%)]	Loss: 1.786274
Train Epoch: 1 [18000/53177 (34%)]	Loss: 1.336032
Train Epoch: 1 [24000/53177 (45%)]	Loss: 1.820539
Train Epoch: 1 [30000/53177 (56%)]	Loss: 1.295821
Train Epoch: 1 [36000/53177 (68%)]	Loss: 1.409304
Train Epoch: 1 [42000/53177 (79%)]	Loss: 1.260964
Train Epoch: 1 [48000/53177 (90%)]	Loss: 1.580474

Average loss: 0.0445, Accuracy: 35980/53177 (67%)

Train Epoch: 2 [6000/53177 (11%)]	Loss: 0.800236
Train Epoch: 2 [12000/53177 (23%)]	Loss: 1.093324